# Chosen ciphertext attack

In this exercise you will write code to demonstrate RSA’s homomorphic property and implement the chosen ciphertext attack.

Use a public key to encrypt a sample message (Em). Then, encrypt a value of r (with r between 2 and 1000). 

Multiply the two numeric versions of the ciphertext together mod n. Multiply the decrypted result (call it D) by the inverse of r (mod n). You should be able to see the original message.

More generally, your program should iterate over all values of r between 2 and 1000 to find the decryption of Em, using the formula: (D / r) % n

In [ ]:
import argparse
import sys
import gmpy2
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key



In [ ]:
def bytes_to_int(text: bytes):
    return int.from_bytes(text, 'big')


def int_to_bytes(num):
    num_int = int(num)
    return int.to_bytes(num_int, (num_int.bit_length()+7)//8, "big")


def encrypt(plain: bytes, e: int, n: int, block_size: int | None = None):
    if block_size is None:
        block_size = (n.bit_length()+7)//8

    blocks = [bytes_to_int(plain[i:i+block_size])
              for i in range(0, len(plain), block_size)]

    return b"".join([int_to_bytes(gmpy2.powmod(block, e, n)) for block in blocks])


def decrypt(cipher: bytes, d: int, n: int, block_size: int | None = None):
    if block_size is None:
        block_size = (n.bit_length()+7)//8

    blocks = [bytes_to_int(cipher[i:i+block_size])
              for i in range(0, len(cipher), block_size)]

    return b"".join([int_to_bytes(gmpy2.powmod(block, d, n)) for block in blocks])

In [ ]:
def main(arguments):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        "-p", "--plaintext", help="File to be encrypted", type=str)
    parser.add_argument(
        "-pub", "--public-key", help="File containing the public key", type=argparse.FileType('r'), required=True)
    parser.add_argument(
        "-prv", "--private-key", help="File containing the private key", type=argparse.FileType('r'), required=True)

    args = parser.parse_args(arguments)

    if args.plaintext:
        original_plain: bytes = args.plaintext.encode()
    else:
        original_plain = b'sample_message'

    public_key = load_pem_public_key(args.public_key.read().encode())
    priv_key = load_pem_private_key(args.private_key.read().encode(), None)

    numbers = public_key.public_numbers()
    encrypted = bytes_to_int(encrypt(original_plain, numbers.e, numbers.n))

    retrieved = []

    for r in range(2, 1001):
        assert (gmpy2.gcd(numbers.n, r) == 1)
        encrypted_r = bytes_to_int(
            encrypt(int_to_bytes(r), numbers.e, numbers.n))

        combined = (encrypted_r*encrypted) % numbers.n

        decrypted_combined = bytes_to_int(decrypt(
            int_to_bytes(combined), priv_key.private_numbers().d, numbers.n))

        inverse_r = pow(r, -1, numbers.n)
        retrieved.append(int_to_bytes(
            decrypted_combined*inverse_r % numbers.n))

    if all(plain == original_plain for plain in retrieved):
        print("All matched")
    else:
        print("Something failed")




In [ ]:


sys.argv = [
    "", 
    "-p", "prime",
    "-pub", "InputFiles/RSA_powmod_without_padding/public_key.pem",
    "-prv" , "InputFiles/RSA_powmod_without_padding/private_key.pem"
]

main(sys.argv[1:])

In [ ]:
with open("InputFiles/RSA_powmod_without_padding/public_key.pem", "r") as pu_key_file:
    print(pu_key_file.read())

In [ ]:
with open("InputFiles/RSA_powmod_without_padding/private_key.pem", "r") as priv_key_file:
    print(priv_key_file.read())